In [ ]:
import numpy as np
import math

# We work in the following field:

K.<phi> = NumberField(x^2 - x - 1)
Q.<i,j,k> = QuaternionAlgebra(K, -1, -1)
Q.variable_names()
a = (1+i+j+k)/2
b = (phi + i*(1/phi) + j)/2


# We define several functions to calculate the presentation of the fundamental group 
# of a given two-bridge knot and of the Dehn surgery

def two_bridge(p, q):
    pres = np.zeros(p*2)
    for i in range(1, p):
        e = (-1)**(math.floor(i*q/p))
        if i%2 == 1:
            if e == 1:
                pres[i-1] = 1
                pres[2*p-1-i] = 3
            else:
                pres[i-1] = 3
                pres[2*p-1-i] = 1
        else:
            if e == 1:
                pres[i-1] = 2
                pres[2*p-1-i] = 4
            else:
                pres[i-1] = 4
                pres[2*p-1-i] = 2
    pres[p-1] = 1
    pres[2*p-1] = 4
    return pres


def convert(pres):
    one = 'f.1'
    two = 'f.2'
    three = 'f.1^-1'
    four = 'f.2^-1'
    times = '*'
    str = ''
    for i in range(0, len(pres)):
        if i > 0:
            str += times
        if pres[i] == 1:
            str += one
        if pres[i] == 2:
            str += two
        if pres[i] == 3:
            str += three
        if pres[i] == 4:
            str += four
    return str


def sigma(p, q):
    sigma = 0
    for i in range(1, p):
        sigma = sigma + (-1)**(math.floor(i*q/p))
    return sigma


def longitude(p, q):
    s = sigma(p,q)
    pres = np.ones(2*s + 2*p - 2)
    pres = 3*pres
    for i in range(1, p):
        e = (-1)**(math.floor(i*q/p))
        if i%2 == 0:
            if e == 1:
                pres[2*s+i-1] = 1
            else:
                pres[2*s+i-1] = 3
        else:
            if e == 1:
                pres[2*s+i-1] = 2
            else:
                pres[2*s+i-1] = 4
    for i in range(1, p):
        e = (-1)**(math.floor(i*q/p))
        if i%2 == 1:
            if e == 1:
                pres[2*s+p-1+i-1] = 1
            else:
                pres[2*s+p-1+i-1] = 3
        else:
            if e == 1:
                pres[2*s+p-1+i-1] = 2
            else:
                pres[2*s+p-1+i-1] = 4
    return pres

def gcd(p,q):
# Create the gcd of two positive integers.
    while q != 0:
        p, q = q, p%q
    return p
def is_coprime(x, y):
    return gcd(x, y) == 1

def second_condition(p, q, n):
    long = longitude(p,q)
    l = len(long)
    pres = np.ones(1+abs(n)*l)
    pres[0] = 1
    if n >= 0:
        for i in range(0, n):
            for j in range(1, l+1):
                pres[i*l + j] = long[j-1]
    else:
        invert_long = np.ones(l)
        for j in range(1, l+1):
            invert_long[l - j] = long[j-1]
        for i in range(0, l):
            if invert_long[i] == 1:
                invert_long[i] = 3
                continue
            if invert_long[i] == 2:
                invert_long[i] = 4
                continue
            if invert_long[i] == 3:
                invert_long[i] = 1
                continue
            if invert_long[i] == 4:
                invert_long[i] = 2
        for i in range(0, abs(n)):
            for j in range(1, l+1):
                pres[i*l + j] = invert_long[j-1]
    return pres


def eval(word):
    value = 1
    while word.find('(') >= 0:
        open = word.find('(')
        close = word.find(')')
        inside = word[open+1:close]
        nexttimes = word[close+1:].find('*')
        if nexttimes >= 0:
            exp = int(word[close+2:close+1+nexttimes])
            rest = word[close+2+nexttimes:]
        else:
            exp = int(word[close+2:])
        word = word[0:open] 
        for i in range (0, exp):
            if i != 0:
                word = word + '*'
            word = word + inside
        if nexttimes >= 0: 
            word = word + '*' + rest
    factors = word.split('*')
    for i in range(0, len(factors)):
        if len(factors[i].strip()) == 1:
            if factors[i].strip() == 'a':
                value = value*a
            else:
                value = value*b
        else:
            newsplit = factors[i].split('^')
            exp = int(newsplit[1])
            if newsplit[0] == 'a':
                value = value*(a**exp)
            else:
                value = value*(b**exp)
    return value

def word_to_coeff(word, coeff, M, N, do):
    for i in range(2, len(word)+1):
        if word[len(word)-i] == 1:
            coeff[0] = 1 + M*coeff[0]
            coeff[1] = M*coeff[1]
            coeff[2] = M*coeff[2]
        if word[len(word)-i] == 2:
            coeff[0] = N*coeff[0]
            coeff[1] = 1 + N*coeff[1]
            coeff[2] = N*coeff[2]
        if word[len(word)-i] == 3:
            coeff[0] = -M^-1 + M^-1*coeff[0]
            coeff[1] = M^-1*coeff[1]
            coeff[2] = M^-1*coeff[2]
        if word[len(word)-i] == 4:
            coeff[0] = N^-1*coeff[0]
            coeff[1] = -N^-1 + N^-1*coeff[1]
            coeff[2] = N^-1*coeff[2]
    return coeff

def check_for_morphism(surj, p, q, n):
    split = surj.split(', ')
    M = eval(split[0])
    N = eval(split[1])
    word = two_bridge(p, q)
    coeff = np.zeros(3)

    if word[len(word)-1] == 1:
        coeff = [1, 0, M]
    if word[len(word)-1] == 2:
        coeff = [0, 1, N]
    if word[len(word)-1] == 3:
        coeff = [-M^-1, 0, M^-1]
    if word[len(word)-1] == 4:
        coeff = [0, -N^-1, N^-1]

    newcoeff = word_to_coeff(word, coeff, M, N, 1)
    a = newcoeff[0]
    b = newcoeff[1]
    
    r_1 = newcoeff[2]
    
    word = second_condition(p, q, n)
    coeff = np.zeros(3)
    if word[len(word)-1] == 1:
        coeff = [1, 0, M]
    if word[len(word)-1] == 2:
        coeff = [0, 1, N]
    if word[len(word)-1] == 3:
        coeff = [-M^-1, 0, M^-1]
    if word[len(word)-1] == 4:
        coeff = [0, -N^-1, N^-1]

    newcoeff = word_to_coeff(word, coeff, M, N, 0)

    c = newcoeff[0]
    d = newcoeff[1]

    r_2 = newcoeff[2]


    if a*d == b*c:
        print("conditions match! there is a mapping for (", p, ",", q, ") and n =", n)
        print("a = ", a, ", b = ", b, ", c = ", c, ", d = ", d)
        print("r_1 = ", r_1, ", r_2 = ", r_2)
    else:
        print("conditions do not match. try again :(")


In [ ]:
for p in range(1, 20): 
    if p % 2 != 0:
        for q in range(2, math.floor(p/2)+1):
            if is_coprime(p, q):
                gap.eval('h := FreeGroup("a", "b");')
                gap.eval('2i := h / [ (h.1*h.2)^2/(h.2^5), (h.1^3)/(h.2^5) ];')
                gap.eval('IsomorphismGroups(2i, SL(2,5));')
                gap.eval('IsomorphismGroups(SL(2,5), 2i);')
                gap.eval('f := FreeGroup("a", "b");')
                finalpres = 'g := f / [ ' + convert(two_bridge(p, q)) + ' ];'
                gap.eval(finalpres)
                newstr = gap.eval('quo := GQuotients(g,2i);')
                if (newstr.find('>')>0):
                    print('For p = ', p, ' and q = ', q, ',\n', newstr)
                    if newstr.find('\n') >= 0:
                        newstr_arr = newstr.split(', \n  ')
                    else:
                        newstr_arr = newstr.split('], [')
                    for i in range(0, len(newstr_arr)):
                        k = newstr_arr[i].find('>')
                        newstr_arr[i] = newstr_arr[i][k+4:]
                        newstr_arr[i] = newstr_arr[i].replace(']', ' ')
                        newstr_arr[i] = newstr_arr[i].rstrip()
                        newstr_arr[i].split(', ')
                        check_for_morphism(newstr_arr[i], p, q, 1)
